In [1]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support import expected_conditions as EC

In [2]:

driver = webdriver.Chrome()  # Ou o navegador que você estiver usando
driver.get("https://consumidor.gov.br/pages/indicador/relatos/abrir")



In [7]:
try:
    # Espera até que o botão de carregar mais comentários esteja presente
    load_more_button = WebDriverWait(driver, 1).until(
        EC.element_to_be_clickable((By.ID , "btn-mais-resultados"))
    )
    load_more_button.click()
except Exception as e:
    print("Erro ao clicar no botão:", e)

In [11]:
titulo_teste = driver.find_elements(By.TAG_NAME , "h3")
titulo_teste[3].text

'Equatorial Maranhão (CEMAR)'

In [19]:
comentarios = WebDriverWait(driver, 5).until(
            EC.presence_of_all_elements_located((By.TAG_NAME, "p"))
        )

comentarios[7].text

'De que maneira será realizado o recadastramento rural?'

In [36]:
tempo_maximo = 1 * 60  # 5 minutos
inicio = time.time()  # Marca o tempo de início

try:
    comentarios_validos = []  # Lista para armazenar comentários válidos
    titulos = []  # Lista para armazenar títulos
    contador = 0
    while True:
        # Verifica se o tempo máximo foi alcançado
        if time.time() - inicio > tempo_maximo:
            print("Tempo máximo de coleta alcançado.")
            break  # Sai do loop se 3 minutos se passaram

        # Coleta os textos dos comentários
        if contador >= 3 and (contador - 3) % 4 == 0:
            comentarios = WebDriverWait(driver, 1).until(
                EC.presence_of_all_elements_located((By.TAG_NAME, "p"))
            )
            comentarios_validos.append(comentarios[contador].text)

        # Coleta os títulos das empresas usando a tag <h3> sem intervalos
        novos_titulos = driver.find_elements(By.TAG_NAME, "h3")
        titulo = novos_titulos[contador].text.strip()
        titulos.append(titulo)
        # titulos.extend([titulo.text.strip() for titulo in novos_titulos])  # Adiciona todos os títulos

        contador += 1

        # # Adiciona comentários válidos à lista
        # for i in range(len(comentarios)):
        #     if i >= 3 and (i - 3) % 4 == 0:  # Mantém os índices relevantes (3, 7, 11, ...)
        #         comentario = comentarios[i].text.strip()
        #         comentarios_validos.append(comentario)

        # Espera até que o botão de carregar mais comentários esteja presente
        try:
            load_more_button = WebDriverWait(driver, 2).until(
                EC.element_to_be_clickable((By.ID, "btn-mais-resultados"))
            )
            load_more_button.click()
            time.sleep(2)  # Espera um pouco para carregar mais comentários
        except Exception as e:
            print("Erro ao clicar no botão ou não há mais comentários:", e)
            break  # Sai do loop se não houver mais comentários

except Exception as e:
    print("Erro ao coletar dados:", e)

Tempo máximo de coleta alcançado.


In [37]:
len(titulos)

19

In [38]:
len(comentarios_validos)

4

In [39]:
titulos

['Cemig',
 'Cemig',
 'Cemig',
 'Equatorial Maranhão (CEMAR)',
 'Cemig',
 'Cemig',
 'Cemig',
 'Cemig',
 'Cemig',
 'Vivo - Telefônica',
 'Equatorial Maranhão (CEMAR)',
 'iFood',
 'iFood',
 'Cemig',
 'Cemig',
 'Cemig',
 'Cemig',
 'Cemig',
 'Cemig']

In [40]:
comentarios_validos

['Preciso do formulário para análise de carga rural.',
 'De que maneira será realizado o recadastramento rural?',
 'Qual é a bandeira tarifária em vigor atualmente e até quando ela permanecerá válida?',
 'Cadastro de tarifa social. NIS: XXX.XXX.XXX-XX Cadastro de baixa renda.']

In [12]:
print(len(titulos))
print(len(comentarios_validos))

29970
29970


In [13]:
df_comentarios = pd.DataFrame({
    'Empresa': titulos,
    'Texto': comentarios_validos
})

In [14]:
df_comentarios[df_comentarios.duplicated(subset=['Texto'])]

,Empresa,Texto
13,Cemig,Qual o percentual do valor da fatura que fica ...
16,Cemig,Qual o Compromisso da Cemig com a sustentabili...
17,Cemig,"O que são esses valores de PIS, COFINS e ICMS ..."
18,Cemig,"O que são esses valores de PIS, COFINS e ICMS ..."
40,Latam Airlines (Tam),"Oi, latam! Tudo bom? Venho aqui suplicar a vcs..."
...,...,...
29956,Serasa Experian,solicito a retirada de meu nome em ofertas de ...
29957,Netflix,Promoção pra ganhar a latinha. Fiquei esperand...
29958,Lojas Colombo,"Boa tarde, fiz um empréstimo com a Colombo, ma..."
29959,Banco Santander,Meu nome consta no SCR e ja eram ter tirado de la


In [16]:
df_comentarios_sem_duplicadas = df_comentarios.drop_duplicates(subset=['Texto'])

In [17]:
df_comentarios_sem_duplicadas

,Empresa,Texto
0,Latam Airlines (Tam),"Oi, latam! Tudo bom? Venho aqui suplicar a vcs..."
1,Azul Linhas Aéreas,Estava olhando algumas passagens e decidi comp...
2,Azul Linhas Aéreas,Gostaria de solicitar a revisão de uma situaçã...
3,Cemig,Qual a documentação para ligação nova?
4,Latam Airlines (Tam),Comprei um voo com a Latam para o dia 01/11 sa...
...,...,...
29965,Serasa Experian,Solicito a exclusão de todas às consultas em m...
29966,Itaú Consignado,"Boa tarde, Prezado venho por meio deste canal ..."
29967,Serasa Experian,Nunca eles retiram a inadimplência do meu nome...
29968,Samsung,Realizei a compra do RELÓGIO DIG INT Watch5 44...


In [18]:
df_comentarios_sem_duplicadas.Empresa.value_counts()

Empresa
Serasa Experian                        126
Latam Airlines (Tam)                    44
Hurb - Hotel Urbano                     37
Cemig                                   35
Equatorial Piauí                        31
                                      ... 
Banese - Banco do Estado de Sergipe      1
Light                                    1
Claro Residencial                        1
SPC Brasil                               1
Bradesco Cartões                         1
Name: count, Length: 167, dtype: int64